# Замер производительности SomeSAT

В данной среде представлен замер производительности реализации алгоритма DPLL, представленной в [репозитории](https://github.com/artem-burashnikov/SomeSAT).

Далее приведены характеристики используемого оборудования и способ подготовки тестового стенда, описание набора тестовых данных, используемые метрики и анализ полученных результатов.

## Характеристики используемого оборудования

Ниже приведены основные характеристики и команды `bash` для их вывода.

1. **Operating system**: Ubuntu 22.04.4 LTS

```bash
grep -E '^PRETTY_NAME=' /etc/os-release
```

2. **Kernel version**: 6.5.0-28-generic

````sh
uname -r
````

3. **RAM**: 7.1Gi

```sh
free -h
```